<a href="https://colab.research.google.com/github/gokulbytes/personalized-news-recommendation-engine/blob/main/main.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Requirements

In [1]:
# Install necessary libraries
!pip install pandas numpy scikit-learn streamlit -q

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.2/10.2 MB 57.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.9/6.9 MB 67.0 MB/s eta 0:00:00


In [2]:
# Import necessary libraries
import pandas as pd
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity
import warnings
warnings.filterwarnings('ignore')

# main

## behaviors_df

In [3]:
# Load the dataset
behaviors_df = pd.read_csv('/content/behaviors.tsv', sep='\t', header=None, names=['UserID', 'TimeStamp', 'History', 'Impressions'])
behaviors_df.head()

,UserID,TimeStamp,History,Impressions
1,U13740,11/11/2019 9:05:58 AM,N55189 N42782 N34694 N45794 N18445 N63302 N104...,N55689-1 N35729-0
2,U91836,11/12/2019 6:11:30 PM,N31739 N6072 N63045 N23979 N35656 N43353 N8129...,N20678-0 N39317-0 N58114-0 N20495-0 N42977-0 N...
3,U73700,11/14/2019 7:01:48 AM,N10732 N25792 N7563 N21087 N41087 N5445 N60384...,N50014-0 N23877-0 N35389-0 N49712-0 N16844-0 N...
4,U34670,11/11/2019 5:28:05 AM,N45729 N2203 N871 N53880 N41375 N43142 N33013 ...,N35729-0 N33632-0 N49685-1 N27581-0
5,U8125,11/12/2019 4:11:21 PM,N10078 N56514 N14904 N33740,N39985-0 N36050-0 N16096-0 N8400-1 N22407-0 N6...


In [4]:
behaviors_df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 156965 entries, 1 to 156965
Data columns (total 4 columns):
 #   Column       Non-Null Count   Dtype 
---  ------       --------------   ----- 
 0   UserID       156965 non-null  object
 1   TimeStamp    156965 non-null  object
 2   History      153727 non-null  object
 3   Impressions  156965 non-null  object
dtypes: object(4)
memory usage: 6.0+ MB


### Drop Duplicates

In [5]:
# Drop duplicate rows and reset the index
behaviors_df = behaviors_df.drop_duplicates().reset_index(drop=True)

## news_df

In [6]:
# Load the dataset
news_df = pd.read_csv('/content/news.tsv', sep='\t', header=None, names=['NewsID', 'Category', 'SubCategory', 'Title', 'Abstract', 'URL','TitleEntities','AbstractEntities'])
news_df.head()

,NewsID,Category,SubCategory,Title,Abstract,URL,TitleEntities,AbstractEntities
0,N55528,lifestyle,lifestyleroyals,"The Brands Queen Elizabeth, Prince Charles, an...","Shop the notebooks, jackets, and more that the...",https://assets.msn.com/labs/mind/AAGH0ET.html,"[{""Label"": ""Prince Philip, Duke of Edinburgh"",...",[]
1,N19639,health,weightloss,50 Worst Habits For Belly Fat,These seemingly harmless habits are holding yo...,https://assets.msn.com/labs/mind/AAB19MK.html,"[{""Label"": ""Adipose tissue"", ""Type"": ""C"", ""Wik...","[{""Label"": ""Adipose tissue"", ""Type"": ""C"", ""Wik..."
2,N61837,news,newsworld,The Cost of Trump's Aid Freeze in the Trenches...,Lt. Ivan Molchanets peeked over a parapet of s...,https://assets.msn.com/labs/mind/AAJgNsz.html,[],"[{""Label"": ""Ukraine"", ""Type"": ""G"", ""WikidataId..."
3,N53526,health,voices,I Was An NBA Wife. Here's How It Affected My M...,"I felt like I was a fraud, and being an NBA wi...",https://assets.msn.com/labs/mind/AACk2N6.html,[],"[{""Label"": ""National Basketball Association"", ..."
4,N38324,health,medical,"How to Get Rid of Skin Tags, According to a De...","They seem harmless, but there's a very good re...",https://assets.msn.com/labs/mind/AAAKEkt.html,"[{""Label"": ""Skin tag"", ""Type"": ""C"", ""WikidataI...","[{""Label"": ""Skin tag"", ""Type"": ""C"", ""WikidataI..."


In [7]:
news_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 51282 entries, 0 to 51281
Data columns (total 8 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   NewsID            51282 non-null  object
 1   Category          51282 non-null  object
 2   SubCategory       51282 non-null  object
 3   Title             51282 non-null  object
 4   Abstract          48616 non-null  object
 5   URL               51282 non-null  object
 6   TitleEntities     51279 non-null  object
 7   AbstractEntities  51278 non-null  object
dtypes: object(8)
memory usage: 3.1+ MB


### Drop Duplicates

In [8]:
# Drop duplicate rows and reset the index
news_df = news_df.drop_duplicates().reset_index(drop=True)

## Data Preprocessing

### behaviors_df

In [9]:
# Fill missing values in the 'History' column with empty strings
behaviors_df['History'].fillna('', inplace=True)

behaviors_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 156965 entries, 0 to 156964
Data columns (total 4 columns):
 #   Column       Non-Null Count   Dtype 
---  ------       --------------   ----- 
 0   UserID       156965 non-null  object
 1   TimeStamp    156965 non-null  object
 2   History      156965 non-null  object
 3   Impressions  156965 non-null  object
dtypes: object(4)
memory usage: 4.8+ MB


### news_df

In [10]:
# Fill missing values in the 'Abstract' column with empty strings
news_df['Abstract'].fillna('', inplace=True)

# Fill missing values in the 'TitleEntities' column with empty strings
news_df['TitleEntities'].fillna('[]', inplace=True)

# Fill missing values in the 'AbstractEntities' column with empty strings
news_df['AbstractEntities'].fillna('[]', inplace=True)

In [11]:
news_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 51282 entries, 0 to 51281
Data columns (total 8 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   NewsID            51282 non-null  object
 1   Category          51282 non-null  object
 2   SubCategory       51282 non-null  object
 3   Title             51282 non-null  object
 4   Abstract          51282 non-null  object
 5   URL               51282 non-null  object
 6   TitleEntities     51282 non-null  object
 7   AbstractEntities  51282 non-null  object
dtypes: object(8)
memory usage: 3.1+ MB


## Knowledge Graph

In [12]:
graph_edges = []

for index, row in news_df.iterrows():
    news_id = row['NewsID']
    category = row['Category']
    subcategory = row['SubCategory']

    # Add edges to Category and SubCategory
    graph_edges.append({'source': news_id, 'relation': 'has_category', 'target': category})
    graph_edges.append({'source': news_id, 'relation': 'has_subcategory', 'target': subcategory})

In [13]:
import json

# Iterate through each row of the news_df DataFrame
for index, row in news_df.iterrows():
    news_id = row['NewsID']
    title_entities_str = row['TitleEntities']
    abstract_entities_str = row['AbstractEntities']

    # Attempt to parse TitleEntities string into a JSON object (list of dictionaries)
    try:
        title_entities = json.loads(title_entities_str)
    except json.JSONDecodeError:
        # If parsing fails (e.g., malformed JSON), default to an empty list
        title_entities = []

    # Attempt to parse AbstractEntities string into a JSON object (list of dictionaries)
    try:
        abstract_entities = json.loads(abstract_entities_str)
    except json.JSONDecodeError:
        # If parsing fails, default to an empty list
        abstract_entities = []

    # Combine all entities from title and abstract into a single list
    all_entities = title_entities + abstract_entities
    # Extract the 'Label' from each entity dictionary if it exists
    entity_labels = [entity['Label'] for entity in all_entities if 'Label' in entity]

    # For each extracted entity label, add an edge to the knowledge graph
    for entity_label in entity_labels:
        graph_edges.append({'source': news_id, 'relation': 'mentions_entity', 'target': entity_label})

In [14]:
from tqdm import tqdm

user_behaviors = {}

# Iterate through each row of the behaviors_df DataFrame with a progress bar
for index, row in tqdm(behaviors_df.iterrows(), total=behaviors_df.shape[0]):
    user_id = row['UserID']
    history = row['History'] # String of space-separated NewsIDs the user has clicked
    impressions = row['Impressions'] # String of space-separated NewsID-clickStatus pairs

    # Split the history string into a list of clicked articles. Handle empty history.
    clicked_articles = history.split() if history else []

    # Split the impressions string into a list of NewsID-clickStatus pairs
    impression_list = impressions.split()
    # Extract NewsIDs that were clicked (ending with '-1')
    clicked_impressions = [imp.split('-')[0] for imp in impression_list if imp.endswith('-1')]
    # Extract NewsIDs that were not clicked (ending with '-0')
    unclicked_impressions = [imp.split('-')[0] for imp in impression_list if imp.endswith('-0')]

    # Store the processed behaviors for each user in the user_behaviors dictionary
    user_behaviors[user_id] = {'clicked_history': clicked_articles, 'clicked_impressions': clicked_impressions, 'unclicked_impressions': unclicked_impressions}

print(f'Processed behaviors for {len(user_behaviors)} users.')

100%|██████████| 156965/156965 [00:17<00:00, 8788.14it/s]

Processed behaviors for 50000 users.


### Embeddings

In [15]:
# Load entity embeddings
entity_embeddings = {}
with open('/content/entity_embedding.vec', 'r') as f:
    next(f)
    for line in f:
        values = line.strip().split('\t')
        entity = values[0]
        embedding = np.array(values[1:], dtype='float32')
        entity_embeddings[entity] = embedding

# Load relation embeddings
relation_embeddings = {}
with open('/content/relation_embedding.vec', 'r') as f:
    next(f)
    for line in f:
        values = line.strip().split('\t')
        relation = values[0]
        embedding = np.array(values[1:], dtype='float32')
        relation_embeddings[relation] = embedding

In [16]:
# Create news embeddings
news_embeddings = {}

# Iterate through each row of the news_df DataFrame with a progress bar
for index, row in tqdm(news_df.iterrows(), total=news_df.shape[0]):
    news_id = row['NewsID']
    title_entities_str = row['TitleEntities']
    abstract_entities_str = row['AbstractEntities']

    # Attempt to parse TitleEntities string into a JSON object
    try:
        title_entities = json.loads(title_entities_str)
    except json.JSONDecodeError:
        title_entities = []

    # Attempt to parse AbstractEntities string into a JSON object
    try:
        abstract_entities = json.loads(abstract_entities_str)
    except json.JSONDecodeError:
        abstract_entities = []

    # Combine all entities from title and abstract
    all_entities = title_entities + abstract_entities
    # Extract Wikidata IDs of entities that exist
    entity_keys = [entity['WikidataId'] for entity in all_entities if 'WikidataId' in entity]

    # Initialize news embedding as a zero vector of the same dimension as entity embeddings
    news_embedding = np.zeros(list(entity_embeddings.values())[0].shape)
    # If there are entities, calculate the news embedding
    if entity_keys:
        # Get embeddings for all valid entity keys
        entity_embeddings_list = [entity_embeddings[key] for key in entity_keys if key in entity_embeddings]
        # If embeddings were found, calculate their mean to represent the news article
        if entity_embeddings_list:
            news_embedding = np.mean(entity_embeddings_list, axis=0)

    # Store the generated news embedding
    news_embeddings[news_id] = news_embedding

print(f'Generated embeddings for {len(news_embeddings)} news articles.')

100%|██████████| 51282/51282 [00:26<00:00, 1904.79it/s]

Generated embeddings for 51282 news articles.


In [17]:
# Create user embeddings
user_embeddings = {}

# Iterate through each user's behaviors with a progress bar
for user_id, behaviors in tqdm(user_behaviors.items(), total=len(user_behaviors)):
    # Combine clicked history and clicked impressions to get all news articles the user clicked
    clicked_articles = behaviors['clicked_history'] + behaviors['clicked_impressions']

    # Initialize user embedding as a zero vector with the same dimension as news embeddings
    user_embedding = np.zeros(list(news_embeddings.values())[0].shape)

    # If the user has clicked any articles
    if clicked_articles:
        # Get the embeddings for all clicked news articles that exist in news_embeddings
        clicked_news_embeddings = [news_embeddings[news_id] for news_id in clicked_articles if news_id in news_embeddings]

        # If valid embeddings were found for clicked articles
        if clicked_news_embeddings:
            # Calculate the user embedding as the mean of the embeddings of their clicked articles
            user_embedding = np.mean(clicked_news_embeddings, axis=0)

    # Store the generated user embedding in the user_embeddings dictionary
    user_embeddings[user_id] = user_embedding

print(f'Generated embeddings for {len(user_embeddings)} users.')

100%|██████████| 50000/50000 [00:39<00:00, 1276.26it/s]

Generated embeddings for 50000 users.


## Recommendation Function

In [18]:
def recommend_news(user_id, news_embeddings, user_embeddings, user_behaviors, news_df, n=10):
    if user_id not in user_embeddings:
        return pd.DataFrame()

    user_embedding = user_embeddings[user_id]
    user_clicked_news = user_behaviors.get(user_id, {}).get('clicked_history', []) + user_behaviors.get(user_id, {}).get('clicked_impressions', [])

    # Calculate similarity scores
    similarity_scores = []
    for news_id, news_embedding in news_embeddings.items():
        if news_id not in user_clicked_news:
            similarity = cosine_similarity([user_embedding], [news_embedding])[0][0]
            similarity_scores.append((news_id, similarity))

    # Sort news articles by similarity score in descending order
    similarity_scores.sort(key=lambda x: x[1], reverse=True)

    # Get top N recommended news articles and their details
    recommended_news_details = []
    for news_id, score in similarity_scores[:n]:
        news_info = news_df[news_df['NewsID'] == news_id].iloc[0]
        recommended_news_details.append({'NewsID': news_id, 'Title': news_info['Title'], 'URL': news_info['URL']})

    return pd.DataFrame(recommended_news_details)

### Recommendations

In [19]:
# Get the first user ID as an example
example_user_id = list(user_embeddings.keys())[0]
recommended_articles = recommend_news(example_user_id, news_embeddings, user_embeddings, user_behaviors, news_df)

print(f'Top 10 recommended articles for user {example_user_id}:')
print(recommended_articles)

Top 10 recommended articles for user U13740:
   NewsID                                              Title  \
0  N22296  N Carolina senator has latched on to Trump. Wi...   
1  N41178  GOP senator defends calling Pelosi 'dumb': 'Wh...   
2  N62021  President Trump to speak at international poli...   
3  N54172  Former U.S. Sen. Kay Hagan, of North Carolina,...   
4  N28476  Former U.S. Sen. Kay Hagan, of North Carolina,...   
5  N13185  Republicans blast Pelosi following vote on imp...   
6  N20107  Supreme Court greenlights Trump asylum ban at ...   
7  N29040  Trump inauguration donor will plead guilty to ...   
8  N35085  House lawmakers vow to secure U.S. visas for S...   
9  N48822  Rep. Brown: President Trump's plan for Syrian ...   

                                             URL  
0  https://assets.msn.com/labs/mind/BBWvHCl.html  
1  https://assets.msn.com/labs/mind/BBWuiYe.html  
2  https://assets.msn.com/labs/mind/AAJaNPw.html  
3  https://assets.msn.com/labs/mind/AAJuLSH.ht

## Save Models

In [20]:
import joblib

# Save user_behaviors dictionary
with open('user_behaviors.pkl', 'wb') as f:
    joblib.dump(user_behaviors, f)
print('User behaviors saved to user_behaviors.pkl')

# Save user embeddings
with open('user_embeddings.pkl', 'wb') as f:
    joblib.dump(user_embeddings, f)
print('User embeddings saved to user_embeddings.pkl')

# Save news embeddings
with open('news_embeddings.pkl', 'wb') as f:
    joblib.dump(news_embeddings, f)
print('News embeddings saved to news_embeddings.pkl')

User behaviors saved to user_behaviors.pkl
User embeddings saved to user_embeddings.pkl
News embeddings saved to news_embeddings.pkl


### Deployment

In [21]:
%%writefile app.py
# Import necessary libraries
import streamlit as st
import pandas as pd
import numpy as np
import joblib
from sklearn.metrics.pairwise import cosine_similarity

# Load data
@st.cache_resource
def load_data():
    with open('user_behaviors.pkl', 'rb') as f:
        user_behaviors = joblib.load(f)
    with open('user_embeddings.pkl', 'rb') as f:
        user_embeddings = joblib.load(f)
    with open('news_embeddings.pkl', 'rb') as f:
        news_embeddings = joblib.load(f)

    news_df = pd.read_csv('/content/news.tsv', sep='\t', header=None, names=['NewsID', 'Category', 'SubCategory', 'Title', 'Abstract', 'URL','TitleEntities','AbstractEntities'])
    news_df['Abstract'].fillna('', inplace=True)
    news_df['TitleEntities'].fillna('[]', inplace=True)
    news_df['AbstractEntities'].fillna('[]', inplace=True)

    return user_behaviors, user_embeddings, news_embeddings, news_df

user_behaviors, user_embeddings, news_embeddings, news_df = load_data()

# RECOMMENDATION FUNCTION
def recommend_news(user_id, news_embeddings, user_embeddings, user_behaviors, news_df, n=10):
    if user_id not in user_embeddings:
        st.warning(f"User ID '{user_id}' not found. Please try another ID.")
        return pd.DataFrame()

    user_embedding = user_embeddings[user_id]
    user_clicked_news = user_behaviors.get(user_id, {}).get('clicked_history', []) + user_behaviors.get(user_id, {}).get('clicked_impressions', [])

    # Calculate similarity scores
    similarity_scores = []
    for news_id, news_embedding in news_embeddings.items():
        if news_id not in user_clicked_news:
            if np.any(user_embedding) and np.any(news_embedding):
                similarity = cosine_similarity([user_embedding], [news_embedding])[0][0]
            else:
                similarity = 0.0
            similarity_scores.append((news_id, similarity))

    # Sort news articles by similarity score in descending order
    similarity_scores.sort(key=lambda x: x[1], reverse=True)

    # Get top N recommended news articles and their details
    recommended_news_details = []
    for news_id, score in similarity_scores[:n]:
        news_info = news_df[news_df['NewsID'] == news_id]
        if not news_info.empty:
            news_info = news_info.iloc[0]
            recommended_news_details.append({'NewsID': news_id, 'Title': news_info['Title'], 'URL': news_info['URL'], 'Similarity Score': score})

    return pd.DataFrame(recommended_news_details)

# STREAMLIT UI
st.title('News Recommendation System')

st.markdown("Enter a User ID to get personalized news recommendations.")

user_id_input = st.text_input('Enter User ID (e.g., U13740)', 'U13740')

if st.button('Get Recommendations'):
    if user_id_input:
        st.subheader(f'Recommendations for User: {user_id_input}')
        recommended_articles_df = recommend_news(user_id_input, news_embeddings, user_embeddings, user_behaviors, news_df)

        if not recommended_articles_df.empty:
            st.dataframe(recommended_articles_df)
        else:
            st.info("Could not retrieve recommendations for this user or user not found.")
    else:
        st.warning("Please enter a User ID.")

Writing app.py
